# Caracterização de eletrofáceis por meio de inteligência artificial com abordagem supervisionada
## Reclassificação da litologia metamorfica em xisto ou filito

- **Nome do Arquivo**: tratamento_dados_metamorfica.ipynb
- **Autor**: Rafael Takeguma Goto
- **Versão**: 1.0
- **Data de Criação**: 11/07/2024
- **Data de Modificação**: 12/07/2024

### Descrição: 
Este notebook contém a etapa do trabalho referente à reclassificação da litologia METAMORFICA NAO IDE em XISTO ou FILITO.

Para reclassificar a metamorfica, será utilizado um modelo de classificação de floresta aleatória, que utilizará as propriedades de perfis dos registros de xisto e filito para aprender e ser capaz de rotular a metamorfica como uma dessas duas litologias.

### Leitura do conjunto de dados

O conjunto de dados é constituído por um arquivo csv, que é resultado da concatenação dos arquivos csv gerados a partir da fusão da extração dos arquivos dlis com os arquivos agp de cada poço, seguida da aplicação dos filtros de constantes, nulos e perfis (DCAL e DRHO).

In [1]:
import pandas as pd

path_pocos_pre_transicao = 'Dados-filtrados-concatenados\pocos_pre_transicao.csv'

dados_concatenados_leitura = pd.read_csv(path_pocos_pre_transicao)

dados_concatenados = pd.DataFrame(dados_concatenados_leitura)

### Separa os registros de XISTO e FILITO
Separa as linhas (registros) em que a litologia é XISTO ou FILITO para treinar o modelo com base nesses dados.

In [2]:
dados_xisto_filito = dados_concatenados[(dados_concatenados['Litologia'] == 'XISTO') | (dados_concatenados['Litologia'] == 'FILITO')]

### Divide os dados em treinamento e teste
Divide os dados armazenados em dados_xisto_filito em 2/3 para treinamento do modelo, e 1/3 para teste do modelo.

In [3]:
from sklearn.model_selection import train_test_split

# Define as variáveis que serão utilizadas (propriedades de perfis) 
propriedades = ['GR', 'DCAL', 'RESD', 'DT', 'RHOB', 'DRHO', 'NPHI', 'PE']

# Separa as variáveis (propriedades de perfis) e armazena em X
X = dados_xisto_filito[propriedades]

# Separa a variável alvo (litologia) e armazena em y
y = dados_xisto_filito['Litologia']

# Divide os dados em 2/3 para treinamento e 1/3 para teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42)

### Treinamento e teste do modelo
Utiliza os dados de treinamento armazenados em X_train e y_train para treinar o modelo de classificação. De forma semelhante, utiliza os dados de teste armazenados em X_test e y_test para testar a acurácia do modelo.

In [4]:
from sklearn.ensemble import RandomForestClassifier

# Treina o modelo
modelo = RandomForestClassifier()
modelo.fit(X_train, y_train)

# Testa o modelo
acuracia = modelo.score(X_test, y_test)
print(f"Acurácia do modelo: {acuracia:2f}")

Acurácia do modelo: 0.991935


### Reclassificação dos registros de metamorfica
Utiliza o modelo treinado acima para prever se os registros que possuem METAMORFICA NAO IDE como litologia tem afinidade para XISTO ou FILITO.

In [5]:
# Separa as linhas (registros) em que a litologia é METAMORFICA NAO IDE 
dados_metamorfica = dados_concatenados[dados_concatenados['Litologia'] == 'METAMORFICA NAO IDE']

# Separa as linhas (registros) da litologia METAMORFICA NAO IDE apenas com as variáveis preditivas
novas_linhas = dados_metamorfica[X.columns]

# Realiza as predições das novas linhas para xisto ou filito
predicoes = modelo.predict(novas_linhas)

# Reclassifica os registros de METAMORFICA NAO IDE em dados_concatenados para XISTO ou FILITO com as predições 
dados_concatenados.loc[dados_concatenados['Litologia'] == 'METAMORFICA NAO IDE', 'Litologia'] = predicoes

In [8]:
teste = pd.DataFrame(predicoes, columns=['Litologia'])

In [21]:
teste_concat = pd.concat([teste, novas_linhas], axis=1, ignore_index=True)

In [22]:
teste_concat

,0,1,2,3,4,5,6,7,8
0,XISTO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XISTO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,XISTO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,XISTO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,XISTO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4285,NaN,134.351364,0.119531,42.540531,76.380360,2.617509,0.018159,0.231577,3.791919
4286,NaN,135.118698,0.109518,42.264965,76.168404,2.600317,0.014432,0.231983,3.781174
4287,NaN,140.874405,0.109518,42.689224,73.448906,2.578244,0.011622,0.221636,3.716026
4288,NaN,139.001068,0.109518,44.850544,72.682953,2.574356,0.008805,0.219713,3.657780


In [9]:
teste

,Litologia
0,XISTO
1,XISTO
2,XISTO
3,XISTO
4,XISTO
...,...
580,XISTO
581,XISTO
582,XISTO
583,XISTO


In [6]:
novas_linhas

,GR,DCAL,RESD,DT,RHOB,DRHO,NPHI,PE
3705,118.836697,0.364842,24.627312,74.633324,2.765261,0.001711,0.161348,3.614490
3706,117.109581,0.359836,26.110756,73.349480,2.781060,0.002619,0.157396,3.591719
3707,120.188148,0.349823,27.706699,72.291008,2.788798,0.002364,0.161401,3.621603
3708,120.672630,0.339810,28.862818,71.034447,2.788282,0.001916,0.161116,3.651869
3709,121.157112,0.329798,30.018936,69.777885,2.787767,0.001468,0.160832,3.682134
...,...,...,...,...,...,...,...,...
4285,134.351364,0.119531,42.540531,76.380360,2.617509,0.018159,0.231577,3.791919
4286,135.118698,0.109518,42.264965,76.168404,2.600317,0.014432,0.231983,3.781174
4287,140.874405,0.109518,42.689224,73.448906,2.578244,0.011622,0.221636,3.716026
4288,139.001068,0.109518,44.850544,72.682953,2.574356,0.008805,0.219713,3.657780


### Novos registros de xisto e filito
Dentre os 585 registros de metamorfica que existiam, 20 foram reclassificados como filito e 565 como xisto.

In [6]:
import numpy as np

# Contagem de ocorrências de cada elemento único
elementos_unicos, contagem = np.unique(predicoes, return_counts=True)

# Exibindo os resultados
for elemento, qtd in zip(elementos_unicos, contagem):
    print(f"{elemento}: {qtd} registro(s)")

FILITO: 26 registro(s)
XISTO: 559 registro(s)


In [9]:
dados_concatenados.to_csv('teste_metamorfica.csv', index=False)

In [7]:
predicoes

array(['XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO',
       'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO',
       'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO',
       'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO',
       'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO',
       'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO',
       'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO',
       'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO',
       'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO',
       'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO',
       'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO',
       'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO',
       'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO',
       'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO', 'XISTO',
       'XISTO', 'XIS